###  Load twitter.json in MongoDB and fetch data

In [1]:
import sys
import json
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

def main():
    "==== Connect to MongoDB ===="
    try:
        client = MongoClient(host="localhost", port=27017)
        twitter1 = client.twitter1
        print("Connected to MongoDB")
    except(ConnectionFailure, e):
        sys.stderr.write("Could not connect to MongoDB %s" % e)
        sys.exit(1)        
    
    with open('corona-out-3', 'r') as f:
        print("Inserting tweets in MongoDB")
        for line in f:
            if(line=='\n'):
                continue            
            tweet = json.loads(line) # load it as Python dict/document for db
            twitter1.tweets.insert_one(tweet)
        print("Completed tweets insert in MongoDB")
        client.close # close db connection        
    f.close() # close file handle     
        
if __name__ == "__main__":
    main()  

Connected to MongoDB
Inserting tweets in MongoDB
Completed tweets insert in MongoDB


In [1]:
import sys
import json
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

def main():
    "==== Connect to MongoDB ===="
    try:
        client = MongoClient(host="localhost", port=27017)
        twitter1 = client.twitter1
        print("Connected to MongoDB")
    except(ConnectionFailure, e):
        sys.stderr.write("Could not connect to MongoDB %s" % e)
        sys.exit(1) 
if __name__ == "__main__":
    main()  

Connected to MongoDB


### Check tweets count

In [2]:
%%bash
mongo twitter1
db.tweets.count()

MongoDB shell version v5.0.6
connecting to: mongodb://127.0.0.1:27017/twitter1?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("2f7f026c-ecae-49b3-bfd5-3ff60e75dc7f") }
MongoDB server version: 4.2.19
which delivers improved usability and compatibility.The "mongo" shell has been deprecated and will be removed in
an upcoming release.
For installation instructions, see
https://docs.mongodb.com/mongodb-shell/install/
101916
bye


### Collection "hashtag"

In [3]:
%%bash
mongo twitter1

db.tweets.mapReduce(  
function() { 
    if(this.entities) { 
        this.entities.hashtags.forEach( function(ele) {
            emit(ele.text, 1)
        })
    }
},
function(key, values) { return Array.sum(values) }, 
{ query: { },  out: "hashtags"}
)

MongoDB shell version v5.0.6
connecting to: mongodb://127.0.0.1:27017/twitter1?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("960434fc-e5bc-4a4e-9c04-2a7a5038b60c") }
MongoDB server version: 4.2.19
which delivers improved usability and compatibility.The "mongo" shell has been deprecated and will be removed in
an upcoming release.
For installation instructions, see
https://docs.mongodb.com/mongodb-shell/install/
{
	"result" : "hashtags",
	"timeMillis" : 8837,
	"counts" : {
		"input" : 101916,
		"emit" : 45119,
		"reduce" : 6068,
		"output" : 7469
	},
	"ok" : 1
}
bye


### What is the 5th most frequent hashtag in the dataset?

In [4]:
%%bash
mongo twitter1
db.hashtags.find().sort({value: -1}).limit(1).skip(4)

MongoDB shell version v5.0.6
connecting to: mongodb://127.0.0.1:27017/twitter1?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("d5897e9b-db95-441d-8d04-490456435adc") }
MongoDB server version: 4.2.19
which delivers improved usability and compatibility.The "mongo" shell has been deprecated and will be removed in
an upcoming release.
For installation instructions, see
https://docs.mongodb.com/mongodb-shell/install/
{ "_id" : "Covid_19", "value" : 973 }
bye


### Collection usermention

In [8]:
%%bash
mongo twitter1
db.tweets.mapReduce(  
function() { 
    if(this.entities) { 
        this.entities.user_mentions.forEach( function(ele) {
            emit(ele.screen_name, 1)
        })
    }
},
function(key, values) { return Array.sum(values) }, 
{ query: { },  out: "usermentions" }
)

MongoDB shell version v5.0.6
connecting to: mongodb://127.0.0.1:27017/twitter1?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("e36b5bc1-1723-4067-a5fe-1afe27b6b456") }
MongoDB server version: 4.2.19
which delivers improved usability and compatibility.The "mongo" shell has been deprecated and will be removed in
an upcoming release.
For installation instructions, see
https://docs.mongodb.com/mongodb-shell/install/
{
	"result" : "usermentions",
	"timeMillis" : 13045,
	"counts" : {
		"input" : 101916,
		"emit" : 100891,
		"reduce" : 22544,
		"output" : 28764
	},
	"ok" : 1
}
{ "_id" : "narendramodi", "value" : 636 }
bye


### What is the 7th most frequent @-mention in the dataset?

In [7]:
%%bash
mongo twitter1
db.usermentions.find().sort({value: -1}).limit(1).skip(6)

MongoDB shell version v5.0.6
connecting to: mongodb://127.0.0.1:27017/twitter1?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("dbdcb770-8af5-4f58-8619-2b1070f5ad3c") }
MongoDB server version: 4.2.19
which delivers improved usability and compatibility.The "mongo" shell has been deprecated and will be removed in
an upcoming release.
For installation instructions, see
https://docs.mongodb.com/mongodb-shell/install/
{ "_id" : "narendramodi", "value" : 636 }
bye


### Collection retweet

In [9]:
%%bash
mongo twitter1

db.tweets.mapReduce(  
function() { 
    if(this.retweeted) { 
        emit(this.user.id, 1)
    }
},
function(key, values) { return Array.sum(values) }, 
{ query: { },  out: "retweets" }
)


MongoDB shell version v5.0.6
connecting to: mongodb://127.0.0.1:27017/twitter1?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("e799e0da-bc7a-4b58-b70c-4447d30a7dd2") }
MongoDB server version: 4.2.19
which delivers improved usability and compatibility.The "mongo" shell has been deprecated and will be removed in
an upcoming release.
For installation instructions, see
https://docs.mongodb.com/mongodb-shell/install/
{
	"result" : "retweets",
	"timeMillis" : 1435,
	"counts" : {
		"input" : 101916,
		"emit" : 0,
		"reduce" : 0,
		"output" : 0
	},
	"ok" : 1
}
bye


### What is the 3th most frequent retweet in the dataset?

In [11]:
%%bash
mongo twitter1
db.retweets.find().sort({retweet_count: -1}).limit(1).skip(2)

MongoDB shell version v5.0.6
connecting to: mongodb://127.0.0.1:27017/twitter1?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("296f6051-9823-48ef-9233-36b7643b3ebf") }
MongoDB server version: 4.2.19
which delivers improved usability and compatibility.The "mongo" shell has been deprecated and will be removed in
an upcoming release.
For installation instructions, see
https://docs.mongodb.com/mongodb-shell/install/
{ "_id" : ObjectId("626fd990d7733d5d0dacb0c4"), "text" : "RT @alokg2k: इंसानों की बस्ती का बस इतना फ़साना है \nखुद को आए तो खांसी, कोई और करे तो corona है । https://t.co/FFgvuDSjYz", "retweeted_status" : { "text" : "इंसानों की बस्ती का बस इतना फ़साना है \nखुद को आए तो खांसी, कोई और करे तो corona है । https://t.co/FFgvuDSjYz", "retweet_count" : 2 } }
bye
